<a href="https://colab.research.google.com/github/Varshiga-tn7/Projects/blob/main/Expiry_Based_Dynamic_Pricing__MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Installation of required libraries***

In [1]:
# Install required libraries
!pip install tensorflow pandas numpy scikit-learn matplotlib seaborn kaggle

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from datetime import datetime


### ***Loading and Displaying Top 5 Rows***

In [2]:
# Load Supermarket Sales Dataset from GitHub
df = pd.read_csv("/content/supermarket_sales - Sheet1.csv")

# Display first 5 rows
df.head()


FileNotFoundError: [Errno 2] No such file or directory: '/content/supermarket_sales - Sheet1.csv'

Preprocessing: Date Conversion & Shelf Life Calculation

In [ ]:
# Convert Date column to datetime format
df["Date"] = pd.to_datetime(df["Date"])

# Assign shelf life based on product type (simulated values)
shelf_life_dict = {
    "Health and beauty": 180,  # 6 months
    "Electronic accessories": 365,  # 1 year
    "Home and lifestyle": 240,  # 8 months
    "Sports and travel": 300,  # 10 months
    "Food and beverages": 90,  # 3 months
    "Fashion accessories": 150  # 5 months
}

# Assign shelf life based on product line
df["Shelf_Life"] = df["Product line"].map(shelf_life_dict)

# Simulate expiry date by adding shelf life to transaction date
df["Expiry_Date"] = df["Date"] + pd.to_timedelta(df["Shelf_Life"], unit='D')

#today = datetime.today()  -> Since data has old values we use the latest transaction as the cuurent date
# Get the latest transaction date in the dataset
latest_transaction_date = df["Date"].max()

# Calculate Days to Expire correctly (Expiry Date - Latest Transaction Date)
df["Days_to_Expire"] = (df["Expiry_Date"] - latest_transaction_date).dt.days

# Ensure there are no negative values (set expired items to 0)
df["Days_to_Expire"] = df["Days_to_Expire"].apply(lambda x: max(x, 0))

# Display results
df[["Product line", "Date", "Expiry_Date", "Days_to_Expire"]].head(10)


### ***Encoding : Assinging Variable for Ease of Use***

In [ ]:
# Encode categorical variables
label_encoder = LabelEncoder()
df["Product_line_encoded"] = label_encoder.fit_transform(df["Product line"])
df["Branch_encoded"] = label_encoder.fit_transform(df["Branch"])
df["City_encoded"] = label_encoder.fit_transform(df["City"])
df["Customer_type_encoded"] = label_encoder.fit_transform(df["Customer type"])
df["Payment_encoded"] = label_encoder.fit_transform(df["Payment"])

df.head()


### ***Feature Scaling using StandardScaler***

In [ ]:
from sklearn.preprocessing import StandardScaler

# Select numerical features for scaling
features = ["Product_line_encoded", "Branch_encoded", "Unit price", "Quantity", "Days_to_Expire"]

scaler = StandardScaler()
df_scaled = df.copy()
df_scaled[features] = scaler.fit_transform(df[features])

df_scaled.head()


### ***Define Target Variable (Discount Prediction)***

In [ ]:
df_scaled["Discount_Percentage"] = np.where(df_scaled["Days_to_Expire"] <= 30,
                                            np.random.uniform(10, 50, df_scaled.shape[0]),
                                            np.random.uniform(0, 10, df_scaled.shape[0]))

# Display data
df_scaled[["Product line", "Days_to_Expire", "Discount_Percentage"]].head()

### ***Train MLP with Different Learning Functions***

In [ ]:
# Import Huber loss
from tensorflow.keras.losses import Huber

# Function to Train MLP with Different Learning Functions
def train_mlp(activation, loss, name):
    model = Sequential([
        Dense(32, activation=activation),
        Dense(16, activation=activation),
        Dense(1, activation="linear")  # Linear for regression models
    ])
    model.compile(optimizer=Adam(learning_rate=0.01), loss=loss, metrics=["mae"])
    model.fit(X_train, y_train, epochs=50, batch_size=8, verbose=0, validation_data=(X_test, y_test))

    # Predict Discounts
    y_pred = model.predict(X).flatten()
    df_scaled[f"Predicted_Discount_{name}"] = y_pred.round(2)

# Train with Different Learning Functions
train_mlp("relu", "mse", "Standard")   # Standard MLP (MSE Loss)
train_mlp("relu", Huber(delta=1.0), "Robust")  # Robust MLP (Huber Loss)
train_mlp("softmax", "categorical_crossentropy", "Probability")  # Probability-based (Softmax Output)
train_mlp("sigmoid", "binary_crossentropy", "Binary")  # Binary Classification (Sigmoid Output)


### ***Compare Performance of Learning Functions***

In [ ]:
# Compare MAE for different learning functions
results = {
    "Standard MLP": mean_absolute_error(y, df_scaled["Predicted_Discount_Standard"]),
    "Robust MLP": mean_absolute_error(y, df_scaled["Predicted_Discount_Robust"]),
    "Probability-Based MLP": mean_absolute_error(y, df_scaled["Predicted_Discount_Probability"]),
    "Binary Classification MLP": mean_absolute_error(y, df_scaled["Predicted_Discount_Binary"])
}

results_df = pd.DataFrame(results, index=["MAE"]).T
print(results_df)


## ***Visualization : Comparison of Actual vs Predicted Discounts***




In [ ]:
plt.figure(figsize=(12,6))

plt.bar(df_scaled["Days_to_Expire"], df_scaled["Discount_Percentage"], label="Actual Discount", alpha=0.5, color="black")
plt.bar(df_scaled["Days_to_Expire"], df_scaled["Predicted_Discount_Standard"], label="Standard MLP", alpha=0.5)
plt.bar(df_scaled["Days_to_Expire"], df_scaled["Predicted_Discount_Robust"], label="Robust MLP", alpha=0.5)
plt.bar(df_scaled["Days_to_Expire"], df_scaled["Predicted_Discount_Probability"], label="Probability-based MLP", alpha=0.5)
plt.bar(df_scaled["Days_to_Expire"], df_scaled["Predicted_Discount_Binary"], label="Binary MLP", alpha=0.5)

plt.xlabel("Days to Expire")
plt.ylabel("Discount Percentage")
plt.title("Predicted vs Actual Discount based on Expiry Date")
plt.legend()
plt.show()


## ***Exporting Results***

In [ ]:
# Save predictions to CSV
df_scaled.to_csv("Predicted_Discounts.csv", index=False)

# Download file
from google.colab import files
files.download("Predicted_Discounts.csv")
